## code sulfate analyzer 

In [2]:
import re, glob, os
from natsort import natsorted

PROTON_MASS = 1.00727647
defaultCharge = -1
absoluteReporterIonIntensityThreshold = 0
neutralLossTransitionRelativeIntensityThreshold = 5
reporterIons = {".SO3-": 79.9573, "HSO3-": 80.9652, ".SO4-": 95.9523, "HSO4-": 96.9601}
ionLosses = {"IonLoss_.SO3-": 79.9573, "IonLoss_HSO3-": 80.9652, "IonLoss_.SO4-": 95.9523, "IonLoss_HSO4-": 96.9601}
neutralLosses = {"NeutralLoss_SO3": 79.9568,"NeutralLoss_H2SO4":97.9674}
ppmErrorThreshold = 10

#exportedSignal = "intensity"
exportedSignal ="normalisedIntensity"

def ppmErrorCalc (observed,expected):
    ppmError = ((observed - expected) / expected) * 1000000
    return ppmError

def normaliseSpectrum (spectrum):
    
    spectralData = spectrum["spectralData"]
    maxIntensity = max(spectralData["intensities"])
    totalIntensity = sum(spectralData["intensities"])
    
    spectrum["spectralData"]["normalisedIntensities"] = []
    normalisedIntensities = []
    
    for i, val in enumerate(spectralData["intensities"]):
        spectrum["spectralData"]["normalisedIntensities"].append(100 * (spectralData["intensities"][i]/maxIntensity))
    
    
    for i, val in enumerate(spectrum["spectrum"]):
        spectrum["spectrum"][i]["normalisedIntensity"] = 100 * (spectrum["spectrum"][i]["intensity"] / maxIntensity)

    spectrum["maxIntensity"] = maxIntensity
    spectrum["totalIntensity"] = totalIntensity
    
    return spectrum

def computeDeltaMasses (spectrum):
    spectralData = spectrum["spectralData"]
    massTransitionDict = {}
    for mass1_i, mass1 in enumerate(spectralData["mzs"]):
        for mass2_i, mass2 in enumerate(spectralData["mzs"]):
            if spectralData["normalisedIntensities"][mass1_i] > neutralLossTransitionRelativeIntensityThreshold and spectralData["normalisedIntensities"][mass2_i] > neutralLossTransitionRelativeIntensityThreshold and mass1 != mass2:
                deltaMass = abs(mass2-mass1)
                #print(deltaMass)
                #masslist = [mass1, mass2]
                intensity1 = '{:.2f}'.format(spectralData["normalisedIntensities"][mass1_i])
                intensity2 = '{:.2f}'.format(spectralData["normalisedIntensities"][mass2_i])
                totalTransitionIntensity = '{:.2f}'.format(spectralData["normalisedIntensities"][mass1_i] + spectralData["normalisedIntensities"][mass2_i])
                #print(totalTransitionIntensity)
                mass1string = '{:.5f}'.format(mass1)
                mass2string = '{:.5f}'.format(mass2)
                
                masstaglist = [mass1string + " (" + intensity1 + ")", mass2string + " (" + intensity2 + ")"]
                
                #print(list(reversed(natsorted(masstaglist))))
                
                #massstringlist = ['{:.5f}'.format(i) for i in masslist] 
                #masslist.sort(reverse=True)
                massTransitionTag = "->".join(list(reversed(natsorted(masstaglist)))) + "[IntensitySum: " + totalTransitionIntensity + "]"
                if massTransitionTag not in massTransitionDict:
                    massTransitionDict[massTransitionTag] = {}
                massTransitionDict[massTransitionTag]["deltaMass"] = deltaMass
            
    spectrum["deltaMasses"] = massTransitionDict
    
    return spectrum
        

def processMGF (mgfFile):
    
    global reporterIons
    reporterIonMatrixReportFile = mgfFile.replace(".mgf", "_RepIonMatrix_" + exportedSignal + ".txt")

    f = open(mgfFile, 'r')

    specID = 0
    spectra = {}
    readingSpectrum = False
    
    for line in f:
        #print(line, end='')
        line = line.rstrip()
        if "BEGIN IONS" in line:
            readingSpectrum = True
            specID += 1
            spectra[specID] = {"title":"", "rt":0, "mz":0, "z":0}
            spectra[specID]["spectrum"] = []
            spectra[specID]["spectralData"] = {}
            spectra[specID]["spectralData"]["mzs"] = []
            spectra[specID]["spectralData"]["intensities"] = []
            spectra[specID]["z"] = 0
            continue

        elif "END IONS" in line:
            spectra[specID]["mw"] = (spectra[specID]["mz"] * abs(spectra[specID]["z"])) - (PROTON_MASS * spectra[specID]["z"])
            spectra[specID]["m"] = (spectra[specID]["mz"] * abs(spectra[specID]["z"]))
            if len(spectra[specID]["spectralData"]["mzs"]) == 0:
                del spectra[specID]
            readingSpectrum = False
            #if specID < 10:
                #print(spectra[specID])

        if readingSpectrum:

            if "TITLE=" in line:
                spectra[specID]["title"] = line.replace("TITLE=","")
                #print(spectra[specID]["title"])
            elif "RTINSECONDS=" in line:
                spectra[specID]["rt"] = float(line.replace("RTINSECONDS=","")) / 60
            elif "PEPMASS=" in line:
                pepmassline = line.replace("PEPMASS=","")
                pepmassfields = pepmassline.split(" ")
                #print(pepmassfields)
                if len(pepmassfields) == 1:
                    precursorIntensity = 0
                else:
                    precursorIntensity = float(pepmassfields[1])

                mz = float(pepmassfields[0])

                spectra[specID]["mz"] = mz

                spectra[specID]["precursorIntensity"] = precursorIntensity
            elif "CHARGE=" in line:
                z = int(line.replace("CHARGE=","").replace("-", ""))
                if "-" in line:
                    spectra[specID]["z"] = -1 * z
                else:
                    spectra[specID]["z"] = z
            else:
                signalLineParts = line.split(" ")
                MS2mz = float(signalLineParts[0])
                MS2intensity = float(signalLineParts[1])
                spectra[specID]["spectrum"].append({'mz': MS2mz, 'intensity': MS2intensity})
                spectra[specID]["spectralData"]["mzs"].append(MS2mz)
                spectra[specID]["spectralData"]["intensities"].append(MS2intensity)
            if spectra[specID]["z"] == 0:
                spectra[specID]["z"] = defaultCharge

    for spec in spectra:
        #print(spec)
        spectra[spec] = normaliseSpectrum(spectra[spec])
        spectra[spec] = computeDeltaMasses(spectra[spec])
        


    matrixReport = ""
    row_id = 0
    
    for spec in spectra:
        row_id += 1
        reporterIons["Precursor"] = spectra[spec]["mz"]

        for n in ionLosses:

            if abs(spectra[spec]["z"]) > 1: 
                daughter_z = abs(spectra[spec]["z"]) - 1 #the charge of the daughter ion will be 1 less than that of the parent
                daughter_m = spectra[spec]["m"] - ionLosses[n] #the mass of the daughter ion will be the mass of the parent ion minus the mass of the ion that is being lost
                daughter_mz = daughter_m / daughter_z #the m/z of the daughter ion from the ion loss
                reporterIons[n] = daughter_mz
                #print(n + ": " + str(daughter_mz))
                
            else:
                reporterIons[n] = 0 

        internalNeutralLossTransitions = []
        totalInternalNeutralLossIntensity = 0
        for n in neutralLosses:

            NL_delta_mz = neutralLosses[n] / abs(spectra[spec]["z"])
            NL_abs_mz = spectra[spec]["mz"] - NL_delta_mz
            reporterIons[n] = NL_abs_mz
            
            for transitionTag, transition in spectra[spec]["deltaMasses"].items():
                ppmError = ppmErrorCalc(transition["deltaMass"], neutralLosses[n])
                if abs(ppmError) < ppmErrorThreshold:
                    internalNeutralLossTransitions.append(n + ": " + transitionTag + "(" + '{:.2f}'.format(ppmError) + " ppm discrepancy)")
        
        matches = re.findall(r'(?<=\[IntensitySum: )[0-9.]+', "|".join(internalNeutralLossTransitions))
        
        for m in matches:
            intensitySum = float(m.replace("[IntensitySum: ", "").replace("]", ""))
            totalInternalNeutralLossIntensity += intensitySum
        
        reporterIonIntensities = dict(zip(list(reporterIons.keys()), [0] * len(reporterIons))) #make a dict filled with zeros
        ionIntensityListStrings = [""] * len(reporterIons)
        if specID < 10:
            print(reporterIons)
            print(reporterIonIntensities)


        #print(spectra[spec])
        spectrum = spectra[spec]["spectrum"]
        #print("Processing spectrum:" + spectra[spec]["title"] + "...")
        #print(spectrum)

        
        for peak in spectrum:

            for reporterName, reporterMZ in reporterIons.items():
                if reporterMZ != 0:
                    ppmError = ppmErrorCalc(peak["mz"], reporterMZ);

                    if abs(ppmError) < ppmErrorThreshold:
                        
                        if peak["intensity"] > absoluteReporterIonIntensityThreshold:
                            #print("reporter ion")
                            reporterIonIntensities[reporterName] = peak[exportedSignal]
                else:
                    reporterIonIntensities[reporterName] = 0
                    
        ionIntensityList = list(reporterIonIntensities.values())
        ionIntensityListStrings = ['{:.6f}'.format(x) for x in ionIntensityList]
        #print(ionIntensityListStrings)
        matches=re.findall(r'\"(.+?)\"',spectra[spec]["title"])
        
        
        if matches[0] == None:
            fileText = ""
        else:
            fileText = matches[0].replace('"', '').replace(',', '')
            
        matrixReport += str(row_id) + "\t" + fileText + "|" + '{:.4f}'.format(spectra[spec]["rt"]) + "|" + '{:.6f}'.format(spectra[spec]["mz"]) + "|" + str(spectra[spec]["z"]) + "\t" + fileText + "\t" + '{:.4f}'.format(spectra[spec]["rt"]) + "\t" + '{:.6f}'.format(spectra[spec]["mw"]) + "\t" + '{:.6f}'.format(spectra[spec]["mz"]) + "\t" + str(spectra[spec]["z"]) + "\t" + str(spectra[spec]["precursorIntensity"]) + "\t" + "\t".join(ionIntensityListStrings)+ "\t" + '{:.1f}'.format(sum(ionIntensityList)-reporterIonIntensities["Precursor"]) + "\t" + '{:.1f}'.format(sum(ionIntensityList)-reporterIonIntensities["Precursor"]+ totalInternalNeutralLossIntensity) + "\t" + '{:.1f}'.format(spectra[spec]["totalIntensity"]) + "\t" + "; ".join(internalNeutralLossTransitions) + "\n"


    matrixReportHeader = "row_id\tAnalyteFullAnnotation\tDataFile\tRT [min]\tMolecular Weight\tm/z\tz\tPrecursor Intensity\t" + "\t".join([h + "_" + exportedSignal for h in list(reporterIons.keys())])+ "\tTotal Reporter Intensity (without INL)\tTotal Reporter Intensity (with INL)\tTotal Intensity\tInternal Neutral Loss Transitions\n"
    matrixReport = matrixReportHeader + matrixReport
    reporterf = open(reporterIonMatrixReportFile, 'w')
    reporterf.write(matrixReport)
    reporterf.close()
    f.close()
    
#os.chdir("/mydir")
for file in glob.glob("*.mgf"):
    print("Processing File: " + file)
    processMGF(file)
    
print("Done!")

Processing File: run1_cent_MS2.mgf.mgf
Processing File: run2_cent_MS2.mgf.mgf
Processing File: run3_cent_MS2.mgf.mgf
Processing File: run4_cent_MS2.mgf.mgf
Processing File: run5_cent_MS2.mgf.mgf
Done!
